In [1]:
!pip install ultralytics

In [5]:
import ultralytics
ultralytics.__version__

'8.2.3'

In [6]:
import numpy as np

In [7]:

import pandas as pd

In [8]:
import cv2

In [9]:
from tracker import*

In [10]:
from ultralytics import YOLO

model=YOLO('yolov8s.pt')

In [12]:
from ultralytics import YOLO

model1=YOLO('yolov8m.pt')

In [11]:
from ultralytics import YOLO

model=YOLO('yolov8n.pt')

RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [13]:
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 
              'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [14]:
tracker=Tracker()
count=0

In [15]:
import time
import os
import json

In [16]:
vehicle_classes = ['car', 'truck', 'motorbike', 'bus']

In [18]:
# Dictionary to store vehicle data
down = {}  # store all cars touching the red line and the locationS
up = {}
counter_down = []  # stores id of all vehicles touching the red line first then the blue line
counter_up = []  # stores id of all vehicles touching the blue line first then the red line

# Create a folder to save frames
if not os.path.exists('detected_frames'):
    os.makedirs('detected_frames')

# Video writer initialization
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (1020, 500))

cap = cv2.VideoCapture('highway.mp4')
count = 0

# Default configuration values
config_file_path = 'config2.json'
last_modified_time = os.path.getmtime(config_file_path)
with open(config_file_path, 'r') as file:
    config = json.load(file)



red_color = (0, 0, 255)  # Red color in BGR
blue_color = (255, 0, 0)  # Blue color in BGR
text_color = (255, 255, 255)  # White color in BGR
green_color = (0, 255, 0)  # Green color in BGR
text_colorcounter = (255, 255, 255)  # White color for text
yellow_color = (0, 255, 255)  # Yellow color for background
counter_bg_color = (0, 0, 255)  # Red color for background
offset = 7

def update_lines(config):
    global green_line_start_x, green_line_start_y, green_line_end_x, green_line_end_y
    global red_line_start_x, red_line_start_y, red_line_end_x, red_line_end_y
    global blue_line_start_x, blue_line_start_y, blue_line_end_x, blue_line_end_y
    
    green_line_start_x = config["speed_test_line"]["green_line_start"]["x"]
    green_line_start_y = config["speed_test_line"]["green_line_start"]["y"]
    green_line_end_x = config["speed_test_line"]["green_line_end"]["x"]
    green_line_end_y = config["speed_test_line"]["green_line_end"]["y"]
    blue_line_start_x = config["speed_test_line"]["blue_line_start"]["x"]
    blue_line_start_y = config["speed_test_line"]["blue_line_start"]["y"]
    blue_line_end_x = config["speed_test_line"]["blue_line_end"]["x"]
    blue_line_end_y = config["speed_test_line"]["blue_line_end"]["y"]
    red_line_start_x = config["red_light_line"]["red_line_start"]["x"]
    red_line_start_y = config["red_light_line"]["red_line_start"]["y"]
    red_line_end_x = config["red_light_line"]["red_line_end"]["x"]
    red_line_end_y = config["red_light_line"]["red_line_end"]["y"]
   

update_lines(config)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    current_modified_time = os.path.getmtime(config_file_path)
    if current_modified_time != last_modified_time:
        with open(config_file_path, 'r') as file:
            config = json.load(file)
        update_lines(config)
        last_modified_time = current_modified_time

    count += 1

    frame = cv2.resize(frame, (1020, 500))

    results = model.predict(frame)
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")
    list = []

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if c in vehicle_classes:
            list.append([x1, y1, x2, y2])

    bbox_id = tracker.update(list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int((x3 + x4) // 2)
        cy = int((y3 + y4) // 2)

        if green_line_start_y < (cy + offset) and green_line_start_y > (cy - offset):
            down[id] = time.time()

        if id in down:
            if blue_line_start_y < (cy + offset) and blue_line_start_y > (cy - offset):
                elapsed_time = (time.time() - down[id])-1
                if counter_down.count(id) == 0:
                    counter_down.append(id)
                    distance = 15
                    a_speed_ms = distance / elapsed_time
                    a_speed_kh = a_speed_ms * 3.6

                    if a_speed_kh > 40:
                        cv2.circle(frame, (cx, cy), 4, red_color, -1)
                        cv2.rectangle(frame, (x3, y3), (x4, y4), red_color, 2)

                        (w, h), _ = cv2.getTextSize(str(int(a_speed_kh)) + 'Km/h', cv2.FONT_HERSHEY_COMPLEX, 0.8, 2)
                        cv2.rectangle(frame, (x4, y4 - h - 10), (x4 + w, y4), red_color, -1)
                        cv2.putText(frame, str(int(a_speed_kh)) + 'Km/h', (x4, y4 - 5), cv2.FONT_HERSHEY_COMPLEX, 0.8, text_color, 2, cv2.LINE_AA)

                        frame_filename = f'detected_frames/{id}.jpg'
                        cv2.imwrite(frame_filename, frame)

        if blue_line_start_y < (cy + offset) and blue_line_start_y > (cy - offset):
            up[id] = time.time()

        if id in up:
            if green_line_start_y < (cy + offset) and green_line_start_y > (cy - offset):
                elapsed1_time = (time.time() - up[id])-1
                if counter_up.count(id) == 0:
                    counter_up.append(id)
                    distance1 = 15
                    a_speed_ms1 = distance1 / elapsed1_time
                    a_speed_kh1 = a_speed_ms1 * 3.6

                    if a_speed_kh1 > 40:
                        cv2.circle(frame, (cx, cy), 4, red_color, -1)
                        cv2.rectangle(frame, (x3, y3), (x4, y4), red_color, 2)
                        (w, h), _ = cv2.getTextSize(str(int(a_speed_kh1)) + 'Km/h', cv2.FONT_HERSHEY_COMPLEX, 0.8, 2)
                        cv2.rectangle(frame, (x4, y4 - h - 10), (x4 + w, y4), red_color, -1)
                        cv2.putText(frame, str(int(a_speed_kh1)) + 'Km/h', (x4, y4 - 5), cv2.FONT_HERSHEY_COMPLEX, 0.8, text_color, 2, cv2.LINE_AA)

                        frame_filename = f'detected_frames/{id}.jpg'
                        cv2.imwrite(frame_filename, frame)

    cv2.line(frame, (green_line_start_x, green_line_start_y), (green_line_end_x, green_line_start_y), green_color, 1)
    cv2.putText(frame, ('Green Line'), (green_line_start_x, green_line_start_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.line(frame, (blue_line_start_x, blue_line_start_y), (blue_line_end_x, blue_line_start_y), blue_color, 1)
    cv2.putText(frame, ('Blue Line'), (blue_line_start_x, blue_line_start_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.line(frame, (red_line_start_x, red_line_start_y), (red_line_end_x, red_line_start_y), red_color, 1)
    cv2.putText(frame, ('Red Line'), (red_line_start_x, red_line_start_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.rectangle(frame, (10, 10), (260, 40), counter_bg_color, -1)
    cv2.putText(frame, ('Going Down - ' + str(len(counter_down))), (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_colorcounter, 2, cv2.LINE_AA)

    cv2.rectangle(frame, (760, 10), (1010, 40), counter_bg_color, -1)
    cv2.putText(frame, ('Going Up - ' + str(len(counter_up))), (765, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_colorcounter, 2, cv2.LINE_AA)

    out.write(frame)
    cv2.imshow("frames", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()




0: 320x640 8 cars, 2 trucks, 136.7ms
Speed: 2.0ms preprocess, 136.7ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 154.7ms
Speed: 3.1ms preprocess, 154.7ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 2 trucks, 128.9ms
Speed: 1.4ms preprocess, 128.9ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 cars, 2 trucks, 120.7ms
Speed: 0.1ms preprocess, 120.7ms inference, 15.8ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 138.6ms
Speed: 11.9ms preprocess, 138.6ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 138.4ms
Speed: 2.1ms preprocess, 138.4ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 127.8ms
Speed: 2.5ms preprocess, 127.8ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 129.5ms
Speed: 1

In [19]:
down = {}  # store all cars touching the red line and the locations
up = {}
counter_down = []  # stores id of all vehicles touching the red line first then the blue line
counter_up = []  # stores id of all vehicles touching the blue line first then the red line

# Create folders to save frames
if not os.path.exists('detected_frames'):
    os.makedirs('detected_frames')

if not os.path.exists('Red_lineviolated_cars'):
    os.makedirs('Red_lineviolated_cars')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (1020, 500))

cap = cv2.VideoCapture('highway.mp4')
count = 0

# Load light configuration
with open('light_status.json', 'r') as filelight:
    configlight = json.load(filelight)
light_status = configlight["light_status"]

# Load line configuration
config_file_path = 'config2.json'
last_modified_time = os.path.getmtime(config_file_path)
with open(config_file_path, 'r') as file:
    config = json.load(file)

red_color = (0, 0, 255)  # Red color in BGR
blue_color = (255, 0, 0)  # Blue color in BGR
text_color = (255, 255, 255)  # White color in BGR
green_color = (0, 255, 0)  # Green color in BGR
text_colorcounter = (255, 255, 255)  # White color for text
yellow_color = (0, 255, 255)  # Yellow color for background
counter_bg_color = (0, 0, 255)  # Red color for background
offset = 7

light_change_time = time.time()

def update_lines(config):
    global green_line_start_x, green_line_start_y, green_line_end_x, green_line_end_y
    global red_line_start_x, red_line_start_y, red_line_end_x, red_line_end_y
    global blue_line_start_x, blue_line_start_y, blue_line_end_x, blue_line_end_y
    
    green_line_start_x = config["speed_test_line"]["green_line_start"]["x"]
    green_line_start_y = config["speed_test_line"]["green_line_start"]["y"]
    green_line_end_x = config["speed_test_line"]["green_line_end"]["x"]
    green_line_end_y = config["speed_test_line"]["green_line_end"]["y"]
    blue_line_start_x = config["speed_test_line"]["blue_line_start"]["x"]
    blue_line_start_y = config["speed_test_line"]["blue_line_start"]["y"]
    blue_line_end_x = config["speed_test_line"]["blue_line_end"]["x"]
    blue_line_end_y = config["speed_test_line"]["blue_line_end"]["y"]
    red_line_start_x = config["red_light_line"]["red_line_start"]["x"]
    red_line_start_y = config["red_light_line"]["red_line_start"]["y"]
    red_line_end_x = config["red_light_line"]["red_line_end"]["x"]
    red_line_end_y = config["red_light_line"]["red_line_end"]["y"]
   

update_lines(config)

# List of vehicle classes to detect


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    current_modified_time = os.path.getmtime(config_file_path)
    if current_modified_time != last_modified_time:
        with open(config_file_path, 'r') as file:
            config = json.load(file)
        update_lines(config)
        last_modified_time = current_modified_time

    count += 1

    frame = cv2.resize(frame, (1020, 500))

    # Change light status every 15 seconds for testing
    if time.time() - light_change_time > 15:
        light_status = "green" if light_status == "red" else "red"
        configlight["light_status"] = light_status
        light_change_time = time.time()
        with open('light_status.json', 'w') as file:
            json.dump(configlight, file, indent=4)

    # Update the red line color based on light status
    current_red_color = green_color if light_status == "green" else red_color

    results = model.predict(frame)
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")
    list = []

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if c in vehicle_classes:
            list.append([x1, y1, x2, y2])

    bbox_id = tracker.update(list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int((x3 + x4) // 2)
        cy = int((y3 + y4) // 2)

        if green_line_start_y < (cy + offset) and green_line_start_y > (cy - offset):
            down[id] = time.time()

        if id in down:
            if blue_line_start_y < (cy + offset) and blue_line_start_y > (cy - offset):
                elapsed_time = (time.time() - down[id])-2
                if counter_down.count(id) == 0:
                    counter_down.append(id)
                    distance = 15
                    a_speed_ms = distance / elapsed_time
                    a_speed_kh = a_speed_ms * 3.6

                    if a_speed_kh > 40:
                        cv2.circle(frame, (cx, cy), 4, red_color, -1)
                        cv2.rectangle(frame, (x3, y3), (x4, y4), red_color, 2)

                        (w, h), _ = cv2.getTextSize(str(int(a_speed_kh)) + 'Km/h', cv2.FONT_HERSHEY_COMPLEX, 0.8, 2)
                        cv2.rectangle(frame, (x4, y4 - h - 10), (x4 + w, y4), red_color, -1)
                        cv2.putText(frame, str(int(a_speed_kh)) + 'Km/h', (x4, y4 - 5), cv2.FONT_HERSHEY_COMPLEX, 0.8, text_color, 2, cv2.LINE_AA)

                        frame_filename = f'detected_frames/{id}.jpg'
                        cv2.imwrite(frame_filename, frame)

        if blue_line_start_y < (cy + offset) and blue_line_start_y > (cy - offset):
            up[id] = time.time()

        if id in up:
            if green_line_start_y < (cy + offset) and green_line_start_y > (cy - offset):
                elapsed1_time = (time.time() - up[id])-2
                if counter_up.count(id) == 0:
                    counter_up.append(id)
                    distance1 = 15
                    a_speed_ms1 = distance1 / elapsed1_time
                    a_speed_kh1 = a_speed_ms1 * 3.6

                    if a_speed_kh1 > 40:
                        cv2.circle(frame, (cx, cy), 4, red_color, -1)
                        cv2.rectangle(frame, (x3, y3), (x4, y4), red_color, 2)
                        (w, h), _ = cv2.getTextSize(str(int(a_speed_kh1)) + 'Km/h', cv2.FONT_HERSHEY_COMPLEX, 0.8, 2)
                        cv2.rectangle(frame, (x4, y4 - h - 10), (x4 + w, y4), red_color, -1)
                        cv2.putText(frame, str(int(a_speed_kh1)) + 'Km/h', (x4, y4 - 5), cv2.FONT_HERSHEY_COMPLEX, 0.8, text_color, 2, cv2.LINE_AA)

                        frame_filename = f'detected_frames/{id}.jpg'
                        cv2.imwrite(frame_filename, frame)
                        
        if light_status == "red":
           if red_line_start_y < (cy + offset) and red_line_start_y > (cy - offset):
                # Check if the car is coming from the left side of the red line
                if cx < red_line_end_x:  # Car is to the left of the red_line_end_x
                  cv2.circle(frame, (cx, cy), 4, red_color, -1)
                  cv2.rectangle(frame, (x3, y3), (x4, y4), red_color, 2)

                  (w, h), _ = cv2.getTextSize('Violate Red', cv2.FONT_HERSHEY_COMPLEX, 0.8, 2)
                  cv2.rectangle(frame, (x4, y4 - h - 10), (x4 + w, y4), red_color, -1)
                  cv2.putText(frame, 'Violate Red', (x4, y4 - 5), cv2.FONT_HERSHEY_COMPLEX, 0.8, text_color, 2, cv2.LINE_AA)
                  frame_filename = f'Red_lineviolated_cars/{id}.jpg'
                  cv2.imwrite(frame_filename, frame)


    cv2.line(frame, (green_line_start_x, green_line_start_y), (green_line_end_x, green_line_start_y), green_color, 1)
    cv2.putText(frame, ('Green Line'), (green_line_start_x, green_line_start_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.line(frame, (blue_line_start_x, blue_line_start_y), (blue_line_end_x, blue_line_start_y), blue_color, 1)
    cv2.putText(frame, ('Blue Line'), (blue_line_start_x + 10, blue_line_start_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.line(frame, (red_line_start_x, red_line_start_y), (red_line_end_x, red_line_start_y), current_red_color, 1)
    cv2.putText(frame, ('Light Status'), (10, red_line_start_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.rectangle(frame, (10, 10), (260, 40), counter_bg_color, -1)
    cv2.putText(frame, ('Going Down - ' + str(len(counter_down))), (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_colorcounter, 2, cv2.LINE_AA)

    cv2.rectangle(frame, (760, 10), (1010, 40), counter_bg_color, -1)
    cv2.putText(frame, ('Going Up - ' + str(len(counter_up))), (765, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_colorcounter, 2, cv2.LINE_AA)

    out.write(frame)
    cv2.imshow("frames", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 320x640 8 cars, 2 trucks, 6212.2ms
Speed: 853.5ms preprocess, 6212.2ms inference, 384.7ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 177.5ms
Speed: 20.4ms preprocess, 177.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 2 trucks, 153.6ms
Speed: 2.0ms preprocess, 153.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 cars, 2 trucks, 141.6ms
Speed: 2.0ms preprocess, 141.6ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 139.6ms
Speed: 13.0ms preprocess, 139.6ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 142.6ms
Speed: 3.0ms preprocess, 142.6ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 148.6ms
Speed: 3.0ms preprocess, 148.6ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 135.6ms
Sp

In [26]:
import os
import cv2
import time
import json
import pandas as pd
from collections import deque

# Store vehicle data
down = {}
up = {}
counter_down = []
counter_up = []

# Create folders to save frames
if not os.path.exists('detected_frames'):
    os.makedirs('detected_frames')

if not os.path.exists('Red_lineviolated_cars'):
    os.makedirs('Red_lineviolated_cars')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (1020, 500))

cap = cv2.VideoCapture('highway.mp4')
count = 0

# Load light configuration
with open('light_status.json', 'r') as filelight:
    configlight = json.load(filelight)
light_status = configlight["light_status"]

# Load line configuration
config_file_path = 'config2.json'
last_modified_time = os.path.getmtime(config_file_path)
with open(config_file_path, 'r') as file:
    config = json.load(file)

# Initialize colors and offset
red_color = (0, 0, 255)
blue_color = (255, 0, 0)
text_color = (255, 255, 255)
green_color = (0, 255, 0)
yellow_color = (0, 255, 255)
counter_bg_color = (0, 0, 255)
offset = 7

light_change_time = time.time()

# Update line positions
def update_lines(config):
    global green_line_start_x, green_line_start_y, green_line_end_x, green_line_end_y
    global red_line_start_x, red_line_start_y, red_line_end_x, red_line_end_y
    global blue_line_start_x, blue_line_start_y, blue_line_end_x, blue_line_end_y

    green_line_start_x = config["speed_test_line"]["green_line_start"]["x"]
    green_line_start_y = config["speed_test_line"]["green_line_start"]["y"]
    green_line_end_x = config["speed_test_line"]["green_line_end"]["x"]
    green_line_end_y = config["speed_test_line"]["green_line_end"]["y"]
    blue_line_start_x = config["speed_test_line"]["blue_line_start"]["x"]
    blue_line_start_y = config["speed_test_line"]["blue_line_start"]["y"]
    blue_line_end_x = config["speed_test_line"]["blue_line_end"]["x"]
    blue_line_end_y = config["speed_test_line"]["blue_line_end"]["y"]
    red_line_start_x = config["red_light_line"]["red_line_start"]["x"]
    red_line_start_y = config["red_light_line"]["red_line_start"]["y"]
    red_line_end_x = config["red_light_line"]["red_line_end"]["x"]
    red_line_end_y = config["red_light_line"]["red_line_end"]["y"]

update_lines(config)

# Vehicle classes to detect
vehicle_classes = ["car", "truck", "bus", "motorcycle"]

# Smoothing mechanism
speed_deque = deque(maxlen=5)  # Store the last 5 speed measurements

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    current_modified_time = os.path.getmtime(config_file_path)
    if current_modified_time != last_modified_time:
        with open(config_file_path, 'r') as file:
            config = json.load(file)
        update_lines(config)
        last_modified_time = current_modified_time

    count += 1

    frame = cv2.resize(frame, (1020, 500))

    # Change light status every 15 seconds for testing
    if time.time() - light_change_time > 15:
        light_status = "green" if light_status == "red" else "red"
        configlight["light_status"] = light_status
        light_change_time = time.time()
        with open('light_status.json', 'w') as file:
            json.dump(configlight, file, indent=4)

    # Update the red line color based on light status
    current_red_color = green_color if light_status == "green" else red_color

    results = model.predict(frame)
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")
    list = []

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if c in vehicle_classes:
            list.append([x1, y1, x2, y2])

    bbox_id = tracker.update(list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int((x3 + x4) // 2)
        cy = int((y3 + y4) // 2)

        if green_line_start_y < (cy + offset) and green_line_start_y > (cy - offset):
            down[id] = time.time()

        if id in down:
            if blue_line_start_y < (cy + offset) and blue_line_start_y > (cy - offset):
                elapsed_time = (time.time() - down[id])-1
                if id not in counter_down:
                    counter_down.append(id)
                    distance = 15  # Correct distance in meters
                    a_speed_ms = distance / elapsed_time
                    a_speed_kh = a_speed_ms * 3.6
                    speed_deque.append(a_speed_kh)  # Add speed to the deque
                    smooth_speed_kh = sum(speed_deque) / len(speed_deque)  # Compute the average speed

                    if smooth_speed_kh > 30:
                        cv2.circle(frame, (cx, cy), 4, red_color, -1)
                        cv2.rectangle(frame, (x3, y3), (x4, y4), red_color, 2)

                        (w, h), _ = cv2.getTextSize(f'{int(smooth_speed_kh)} Km/h', cv2.FONT_HERSHEY_COMPLEX, 0.8, 2)
                        cv2.rectangle(frame, (x4, y4 - h - 10), (x4 + w, y4), red_color, -1)
                        cv2.putText(frame, f'{int(smooth_speed_kh)} Km/h', (x4, y4 - 5), cv2.FONT_HERSHEY_COMPLEX, 0.8, text_color, 2, cv2.LINE_AA)

                        frame_filename = f'detected_frames/{id}.jpg'
                        cv2.imwrite(frame_filename, frame)

        if blue_line_start_y < (cy + offset) and blue_line_start_y > (cy - offset):
            up[id] = time.time()

        if id in up:
            if green_line_start_y < (cy + offset) and green_line_start_y > (cy - offset):
                elapsed1_time = (time.time() - up[id])-1
                if id not in counter_up:
                    counter_up.append(id)
                    distance1 = 15  # Correct distance in meters
                    a_speed_ms1 = distance1 / elapsed1_time
                    a_speed_kh1 = a_speed_ms1 * 3.6
                    speed_deque.append(a_speed_kh1)  # Add speed to the deque
                    smooth_speed_kh1 = sum(speed_deque) / len(speed_deque)  # Compute the average speed

                    if smooth_speed_kh1 > 30:
                        cv2.circle(frame, (cx, cy), 4, red_color, -1)
                        cv2.rectangle(frame, (x3, y3), (x4, y4), red_color, 2)

                        (w, h), _ = cv2.getTextSize(f'{int(smooth_speed_kh1)} Km/h', cv2.FONT_HERSHEY_COMPLEX, 0.8, 2)
                        cv2.rectangle(frame, (x4, y4 - h - 10), (x4 + w, y4), red_color, -1)
                        cv2.putText(frame, f'{int(smooth_speed_kh1)} Km/h', (x4, y4 - 5), cv2.FONT_HERSHEY_COMPLEX, 0.8, text_color, 2, cv2.LINE_AA)

                        frame_filename = f'detected_frames/{id}.jpg'
                        cv2.imwrite(frame_filename, frame)

        if light_status == "red":
            if red_line_start_y < (cy + offset) and red_line_start_y > (cy - offset):
                if cx < red_line_end_x:
                    cv2.circle(frame, (cx, cy), 4, red_color, -1)
                    cv2.rectangle(frame, (x3, y3), (x4, y4), red_color, 2)

                    (w, h), _ = cv2.getTextSize('Violate Red', cv2.FONT_HERSHEY_COMPLEX, 0.8, 2)
                    cv2.rectangle(frame, (x4, y4 - h - 10), (x4 + w, y4), red_color, -1)
                    cv2.putText(frame, 'Violate Red', (x4, y4 - 5), cv2.FONT_HERSHEY_COMPLEX, 0.8, text_color, 2, cv2.LINE_AA)
                    frame_filename = f'Red_lineviolated_cars/{id}.jpg'
                    cv2.imwrite(frame_filename, frame)

    cv2.line(frame, (green_line_start_x, green_line_start_y), (green_line_end_x, green_line_start_y), green_color, 1)
    cv2.putText(frame, 'Green Line', (green_line_start_x, green_line_start_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.line(frame, (blue_line_start_x, blue_line_start_y), (blue_line_end_x, blue_line_start_y), blue_color, 1)
    cv2.putText(frame, 'Blue Line', (blue_line_start_x + 10, blue_line_start_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.line(frame, (red_line_start_x, red_line_start_y), (red_line_end_x, red_line_start_y), current_red_color, 1)
    cv2.putText(frame, 'Light Status', (10, red_line_start_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.rectangle(frame, (10, 10), (260, 40), counter_bg_color, -1)
    cv2.putText(frame, 'Going Down - ' + str(len(counter_down)), (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_colorcounter, 2, cv2.LINE_AA)

    cv2.rectangle(frame, (760, 10), (1010, 40), counter_bg_color, -1)
    cv2.putText(frame, 'Going Up - ' + str(len(counter_up)), (765, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_colorcounter, 2, cv2.LINE_AA)

    out.write(frame)
    cv2.imshow("frames", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 320x640 8 cars, 2 trucks, 139.6ms
Speed: 1.0ms preprocess, 139.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 151.6ms
Speed: 2.0ms preprocess, 151.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 2 trucks, 136.6ms
Speed: 3.0ms preprocess, 136.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 cars, 2 trucks, 127.7ms
Speed: 3.0ms preprocess, 127.7ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 128.7ms
Speed: 3.0ms preprocess, 128.7ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 136.6ms
Speed: 3.0ms preprocess, 136.6ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 131.6ms
Speed: 2.0ms preprocess, 131.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 134.6ms
Speed: 2.0